In [1]:
import os
import json
import pandas as pd
import pickle
import cv2


from tqdm import tqdm
from datetime import datetime

In [2]:
with open(os.path.join('/data/chanwutk/code/apperception/outputs/segment-tracking-without-de-segment.json'), 'r') as f:
    trajectory_json = json.load(f)
trajectory_json[[*trajectory_json.keys()][0]][4]

{'1': {'detection_id': [4, 0],
  'car_loc3d': [623.2197228094511, 1630.1267423144393, 0.0],
  'timestamp': '2018-08-01 12:26:43.862573',
  'segment_line': '010200000002000000E00AD25DEABA834050C53524C16E9940E00AD25DEABE834050C53524C1709940',
  'segment_heading': -0.7853982,
  'road_polygon_info': {'id': '153a9e49-ac09-4c83-a01c-d16a831e6b3c',
   'polygon': 'POLYGON ((620.7369120892475 1621.0347849185798, 622.1562563964527 1622.4810413880725, 622.9290287821966 1623.3643432444885, 623.9665777304992 1624.7068729164082, 623.9530934200651 1625.3024886261435, 623.5260647458696 1626.3622680096885, 622.9358509123862 1627.0974895587487, 621.5599555757373 1628.279739826948, 620.4964859313457 1629.1462563400924, 619.9468333576316 1629.4562711643039, 620.3705202615148 1631.016220449493, 623.3077336112881 1634.4064593753496, 626.1456150616126 1637.486664008812, 627.2572457310254 1638.5643203340883, 631.1438044179225 1635.189459658185, 632.1235249140449 1634.9461365883333, 632.9177055439161 1635.0105

In [3]:
with open(os.path.join('/data/chanwutk/code/apperception/outputs/segment-tracking-without-de-3d.json'), 'r') as f:
    trajectory_point_json = json.load(f)

In [4]:
with open('/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/videos/frames.pkl', 'rb') as f:
    frames = pickle.load(f)
frames['scene-0061-CAM_FRONT']

{'filename': 'singapore-onenorth-scene-0061-CAM_FRONT.mp4',
 'start': datetime.datetime(2018, 7, 23, 20, 28, 47, 612460),
 'location': 'singapore-onenorth',
 'columns': ['cameraId',
  'frameId',
  'frameNum',
  'filename',
  'cameraTranslation',
  'cameraRotation',
  'cameraIntrinsic',
  'egoTranslation',
  'egoRotation',
  'timestamp',
  'cameraHeading',
  'egoHeading',
  'location'],
 'frames': [('scene-0061',
   'e3d495d4ac534d54b321f50006683844',
   2,
   'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
   array([ 410.87244105, 1179.57081167,    1.49367752]),
   array([-0.12889417,  0.1153416 ,  0.70315973, -0.68967311]),
   [[1266.417203046554, 0.0, 816.2670197447984],
    [0.0, 1266.417203046554, 491.50706579294757],
    [0.0, 0.0, 1.0]],
   [411.4199861830012, 1181.197175631848, 0.0],
   [0.5720063498929273,
    -0.0021434844534272707,
    0.011564094980151613,
    -0.8201648693182716],
   datetime.datetime(2018, 7, 23, 20, 28, 47, 612460),
   

In [5]:
def build_trajectories(metadata):
    id_to_trajectory = {}
    for m in metadata:
        for oid, det in m.items():
            if oid not in id_to_trajectory:
                id_to_trajectory[oid] = []
            id_to_trajectory[oid].append(det)
    
    for dets in id_to_trajectory.values():
        dets.sort(key=lambda d: d['timestamp'])
    return id_to_trajectory

In [ ]:
video_output_path = '/data/chanwutk/code/apperception/outputs/videos'

if not os.path.exists(video_output_path):
    os.makedirs(video_output_path)

for scene_name, trajs in trajectory_json.items():
    print(scene_name)
    id_to_trajectory = build_trajectories(trajs)
    _frames = frames[scene_name]
    video_filename = _frames['filename']
    cap = cv2.VideoCapture(os.path.join('/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/videos', video_filename))
    
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    video_frames = []
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            video_frames.append(frame)
        else: 
            break
    
    assert len(video_frames) == len(_frames['frames'])

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    
    video_scene_path = os.path.join(video_output_path, scene_name)
    if not os.path.exists(video_scene_path):
        os.makedirs(video_scene_path)
    
    for oid, trajectory in id_to_trajectory.items():
        min_time = min(t['timestamp'] for t in trajectory)
        max_time = max(t['timestamp'] for t in trajectory)
        print(oid, min_time, max_time)
        filtered_frames = [(i, f) for i, f in enumerate(_frames['frames']) if min_time <= str(f[9]) <= max_time]
#         print(trajectory[0])
#         print([str(i) + ' ' + str(f[9]) for i, f in filtered_frames])
    
        out = cv2.VideoWriter(os.path.join(video_scene_path, f'{oid}.mp4'),cv2.VideoWriter_fourcc(*'mp4v'), 12, (1600, 900))
        for i, f in tqdm(filtered_frames):
            video_frame = video_frames[i].copy()
            segments = [s for s in trajectory if s['detection_id'][0] == i]
            assert len(segments) <= 1, segments
            if len(segments) == 0:
                assert oid not in trajectory_point_json[scene_name][i]
            else:
                segment = segments[0]
                det = trajectory_point_json[scene_name][i][oid]
                assert det['detection_id'] == segment['detection_id']
                assert det['detection_id'][0] == i
                
                start_point = (det['bbox_left'], det['bbox_top'])
                end_point = (det['bbox_left'] + det['bbox_w'], det['bbox_top'] + det['bbox_h'])
                color = (255, 0, 0)
                thickness = 2
                video_frame = cv2.rectangle(video_frame, start_point, end_point, color, thickness)
            out.write(video_frame)
        out.release()


scene-0103-CAM_FRONT
1 2018-08-01 12:26:43.662404 2018-08-01 12:27:02.912404


  0%|                                                                                                                                                                                                                                                 | 0/227 [00:00<?, ?it/s]